In [64]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [65]:
# Import some of the most important libary for using this notebook
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


In [66]:
file_path = 'Data.csv'
# Read csv file for using pandas libary
sarcasm_df = pd.read_csv(file_path)
# Looked first five rows

label_mapping = {'Sarcastic': 1, 'Non Sarcastic': 0}
sarcasm_df['target'] = sarcasm_df['target'].map(label_mapping)
sarcasm_df.head()

,headlines,target
0,CNN Triumphs (At Least in Most Demographic Cat...,0
1,"‘You Did The Best You Could,’ Says Iron Man Ac...",1
2,New Emails Reveal Warm Relationship Between Ka...,0
3,Donald Trump Jr. Gets Slammed Over Racist Birt...,0
4,God Urges Rick Perry Not To Run For President,1


In [67]:
vocab_size = 10000 # It's most common vocabulary size.
embedding_dim = 16 
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

In [68]:
# Every word map to unique number
max_words = 10000
max_sequence_length = 100

# Tokenize headlines
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(sarcasm_df['headlines'])
sequences = tokenizer.texts_to_sequences(sarcasm_df['headlines'])

# Pad sequences to ensure uniform input length
X = pad_sequences(sequences, maxlen=max_sequence_length)
y = sarcasm_df['target'].values

In [69]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [70]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((10004, 100), (2502, 100), (10004,), (2502,))

In [71]:
# Create a Callback for using when I train model.
e_callbacks = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=2,
    restore_best_weights=True,
)
bm_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='/kaggle/working/best_model.keras',
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=False  # If you want to save the full model, not just the weights
)


In [88]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_sequence_length))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(64))
model.add(Dense(1, activation='sigmoid'))  # Binary classification

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=32)

Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 27s 74ms/step - accuracy: 0.7593 - loss: 0.4369 - val_accuracy: 0.9005 - val_loss: 0.2331
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 22s 70ms/step - accuracy: 0.9691 - loss: 0.0967 - val_accuracy: 0.9041 - val_loss: 0.2557
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 22s 71ms/step - accuracy: 0.9879 - loss: 0.0425 - val_accuracy: 0.9045 - val_loss: 0.3288
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 22s 72ms/step - accuracy: 0.9928 - loss: 0.0218 - val_accuracy: 0.8949 - val_loss: 0.4099
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 19s 62ms/step - accuracy: 0.9939 - loss: 0.0181 - val_accuracy: 0.8969 - val_loss: 0.5045


In [90]:
model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_8 (Embedding)         │ (None, 100, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_14 (LSTM)                  │ (None, 100, 64)        │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_15 (LSTM)                  │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,087,493 (15.59 MB)

 Trainable params: 1,362,497 (5.20 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,724,996 (10.40 MB)

In [91]:
model.save('sarcasm_detect.h5')

In [92]:
import tensorflow as tf
load_model  = tf.keras.models.load_model('sarcasm_detect.h5')
load_model.evaluate(X_test,tf.expand_dims(y_test,axis=1))

79/79 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.9097 - loss: 0.4386


[0.5044990181922913, 0.8968824744224548]

In [93]:
for i,input_text_with_label in enumerate(zip(X_test[50:],y_test[50:])):
    if i > 5:
        break
    else:
        test_input_seq = input_text_with_label[0]
        actual_label = input_text_with_label[1]
        
        probs = np.squeeze(load_model.predict(np.array([test_input_seq])))
        pred = int(np.round(np.max(probs)))
        print(f"Prediction Value Is: {pred}")
        print(f'Actual Value Is: {actual_label}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step
Prediction Value Is: 0
Actual Value Is: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Prediction Value Is: 1
Actual Value Is: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Prediction Value Is: 0
Actual Value Is: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Prediction Value Is: 1
Actual Value Is: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Prediction Value Is: 0
Actual Value Is: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Prediction Value Is: 1
Actual Value Is: 1
